In [ ]:
!pip install -q transformers accelerate bitsandbytes langchain_community

In [ ]:
!pip install chromadb

In [ ]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [ ]:
!unzip -q new_articles.zip -d new_articles

In [ ]:
loader = DirectoryLoader("/content/new_articles/", glob = "./*.txt", loader_cls= TextLoader)
document=loader.load()

In [ ]:
import os
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(document)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
api_key = ""  # Replace with your actual key

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "tiiuae/falcon-7b-instruct"  # or try smaller like "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.7)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.vectorstores import Chroma

persist_directory = "db"
vectordb = Chroma.from_documents(documents=docs, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()

# Reload if needed later
# vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
query = "How much money did Microsoft raise?"
response = qa_chain(query)

# Output result with source
def process_llm_response(res):
    print("Answer:", res['result'])
    print("\nSources:")
    for src in res["source_documents"]:
        print(src.metadata["source"])

process_llm_response(response)